In [62]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

accre = pd.read_csv('data/accre_data.txt', sep='|', nrows=5000)
accre.head()

,JobID,JobIDRaw,Cluster,Partition,Account,Group,GID,User,UID,Submit,...,State,NNodes,NCPUS,ReqCPUS,ReqMem,ReqGRES,ReqTRES,Timelimit,NodeList,JobName
0,15806649,15806649,accre,production,h_vuiis,h_vuiis,36052,masispider,NaN,2017-07-02T00:40:58,...,COMPLETED,1,2,2,12Gn,NaN,"cpu=2,mem=12G,node=1",9-04:00:00,vmp1085,WOODWARD_TCP-x-228324-x-228324-x-301-x-Multi_A...
1,15806664,15806664,accre,production,h_vuiis,h_vuiis,36052,masispider,NaN,2017-07-02T00:40:58,...,COMPLETED,1,2,2,12Gn,NaN,"cpu=2,mem=12G,node=1",9-04:00:00,vmp1099,WOODWARD_TCP-x-115174-x-115174-x-301-x-Multi_A...
2,15806670,15806670,accre,production,h_vuiis,h_vuiis,36052,masispider,NaN,2017-07-02T00:40:58,...,COMPLETED,1,2,2,12Gn,NaN,"cpu=2,mem=12G,node=1",9-04:00:00,vmp1126,WOODWARD_TCP-x-215607-x-215607-x-301-x-Multi_A...
3,15806671,15806671,accre,production,h_vuiis,h_vuiis,36052,masispider,NaN,2017-07-02T00:40:58,...,COMPLETED,1,2,2,12Gn,NaN,"cpu=2,mem=12G,node=1",9-04:00:00,vmp230,WOODWARD_TCP-x-116214-x-116214-x-1701-x-Multi_...
4,15811617,15811617,accre,production,cms_samtest,cms_samtest,59297,uscms010,NaN,2017-07-01T05:00:00,...,COMPLETED,1,1,1,4000Mn,NaN,"cpu=1,mem=4000M,node=1",2-00:00:00,vmp1242,bl_aa7c22e04f21


In [63]:
accre.shape

(5000, 25)

In [64]:
accre.columns

Index(['JobID', 'JobIDRaw', 'Cluster', 'Partition', 'Account', 'Group', 'GID',
       'User', 'UID', 'Submit', 'Eligible', 'Start', 'End', 'Elapsed',
       'ExitCode', 'State', 'NNodes', 'NCPUS', 'ReqCPUS', 'ReqMem', 'ReqGRES',
       'ReqTRES', 'Timelimit', 'NodeList', 'JobName'],
      dtype='object')

In [65]:
accre.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 25 columns):
JobID        5000 non-null object
JobIDRaw     5000 non-null int64
Cluster      5000 non-null object
Partition    5000 non-null object
Account      5000 non-null object
Group        5000 non-null object
GID          5000 non-null int64
User         5000 non-null object
UID          3821 non-null float64
Submit       5000 non-null object
Eligible     5000 non-null object
Start        5000 non-null object
End          5000 non-null object
Elapsed      5000 non-null object
ExitCode     5000 non-null object
State        5000 non-null object
NNodes       5000 non-null int64
NCPUS        5000 non-null int64
ReqCPUS      5000 non-null int64
ReqMem       5000 non-null object
ReqGRES      13 non-null object
ReqTRES      5000 non-null object
Timelimit    5000 non-null object
NodeList     5000 non-null object
JobName      5000 non-null object
dtypes: float64(1), int64(5), object(19)
memory u

In [68]:
accre['Start_Column'] = accre['Start'].str.split('T')
accre['Start_Column'].head()

0    [2017-07-03, 02:13:59]
1    [2017-07-03, 02:20:08]
2    [2017-07-03, 02:41:13]
3    [2017-07-03, 02:51:53]
4    [2017-07-01, 05:00:01]
Name: Start_Column, dtype: object

In [73]:
accre['End_Column'] = accre['End'].str.split('T')

In [75]:
accre['Submit_Column'] = accre['Submit'].str.split('T')

In [76]:
accre['Start_Date'] = [loc[0] for loc in accre['Start_Column']]

In [77]:
accre['Start_Time'] = [loc[1] for loc in accre['Start_Column']]

In [78]:
accre['End_Date'] = [loc[0] for loc in accre['End_Column']]

In [79]:
accre['End_Time'] = [loc[1] for loc in accre['End_Column']]

In [83]:
accre['Submit_Date'] = [loc[0] for loc in accre['Submit_Column']]

In [84]:
accre['Submit_Time'] = [loc[1] for loc in accre['Submit_Column']]

In [85]:
accre_subset = accre.loc[:,["JobID","Partition","Account","Group","User","Submit_Date","Submit_Time","Start_Date","Start_Time","End_Date","End_Time","Elapsed","State","Nnodes","NCPUS","ReqCPUS","ReqMem","Timelimit"]]
accre_subset.head()

/Users/davisthrailkill/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


,JobID,Partition,Account,Group,User,Submit_Date,Submit_Time,Start_Date,Start_Time,End_Date,End_Time,Elapsed,State,Nnodes,NCPUS,ReqCPUS,ReqMem,Timelimit
0,15806649,production,h_vuiis,h_vuiis,masispider,2017-07-02,00:40:58,2017-07-03,02:13:59,2017-07-04,03:19:20,1-01:05:21,COMPLETED,NaN,2,2,12Gn,9-04:00:00
1,15806664,production,h_vuiis,h_vuiis,masispider,2017-07-02,00:40:58,2017-07-03,02:20:08,2017-07-04,02:11:55,23:51:47,COMPLETED,NaN,2,2,12Gn,9-04:00:00
2,15806670,production,h_vuiis,h_vuiis,masispider,2017-07-02,00:40:58,2017-07-03,02:41:13,2017-07-04,04:41:04,1-01:59:51,COMPLETED,NaN,2,2,12Gn,9-04:00:00
3,15806671,production,h_vuiis,h_vuiis,masispider,2017-07-02,00:40:58,2017-07-03,02:51:53,2017-07-04,10:54:13,1-08:02:20,COMPLETED,NaN,2,2,12Gn,9-04:00:00
4,15811617,production,cms_samtest,cms_samtest,uscms010,2017-07-01,05:00:00,2017-07-01,05:00:01,2017-07-01,05:00:11,00:00:10,COMPLETED,NaN,1,1,4000Mn,2-00:00:00
